In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
from edward.models import Categorical, Normal
import edward as ed
import pandas as pd

learning_rate = 0.001
batch_size = 128
n_epochs = 25

mnist = input_data.read_data_sets('/data/mnist', one_hot=True) 
#create placeholders for features and labels
X = tf.placeholder(dtype=tf.float32, shape=(batch_size,784), name="X")
Y_plh = tf.placeholder(dtype=tf.int32, shape=(batch_size,10), name="Y")

#weights, biases
W = Normal(loc=tf.zeros([784, 10]), scale=tf.ones(784, 10))
b = Normal(loc=tf.zeros(10),scale=tf.ones(10))

#create logits
Y = Categorical(tf.matmul(X,W)+b)

#create loss
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name="loss")
loss = tf.reduce_mean(entropy) # compute mean of all examples in batch

#create prior over weights
qw = Normal(loc=tf.Variable(tf.random_normal([784,10])),scale=tf.nn.softplus(tf.Variable(tf.random_normal([784,10]))))
qb = Normal(loc=tf.Variable(tf.random_normal([10])),scale=tf.nn.softplus(tf.Variable(tf.random_normal([10]))))

inference = ed.KLqp({W:qw,b:qb}, data={Y:Y_ph})

inference.initialize(n_iter=5000, n_print=100, scale={y:float(55000/100)})

with ed.get_session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./graphs/logistic_reg', sess.graph)
    start_time = time.time()
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs):
        total_loss = 0
        for j in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            Y_batch = np.argmax(Y_batch,axis=1)
            info_dict = inference.update(feed_dict={x: X_batch, y_ph: Y_batch})
            inference.print_progress(info_dict)

    print('Total time: {0} seconds'.format(time.time() - start_time))
    
    X_test = mnist.test.images
    Y_test = np.argmax(mnist.test.labels,axis=1)

    n_samples = 100
    prob_lst = []
    samples = []
    w_samples = []
    b_samples = []
    for _ in range(n_samples):
        w_samp = qw.sample()
        b_samp = qb.sample()
        w_samples.append(w_samp)
        b_samples.append(b_samp)
        # Also compue the probabiliy of each class for each (w,b) sample.
        prob = tf.nn.softmax(tf.matmul( X_test,w_samp ) + b_samp)
        prob_lst.append(prob.eval())
        sample = tf.concat([tf.reshape(w_samp,[-1]),b_samp],0)
        samples.append(sample.eval())

    accy_test = []
    for prob in prob_lst:
        y_trn_prd = np.argmax(prob,axis=1).astype(np.float32)
        acc = (y_trn_prd == Y_test).mean()*100
        accy_test.append(acc)

    # Load the first image from the test data and its label.
    test_image = X_test[0:1]
    test_label = Y_test[0]
    print('truth = ',test_label)
    pixels = test_image.reshape((28, 28))
    plt.imshow(pixels,cmap='Blues')

    # Now the check what the model perdicts for each (w,b) sample from the posterior. This may take a few seconds...
    sing_img_probs = []
    for w_samp,b_samp in zip(w_samples,b_samples):
        prob = tf.nn.softmax(tf.matmul( X_test[0:1],w_samp ) + b_samp)
        sing_img_probs.append(prob.eval())

    plt.hist(np.argmax(sing_img_probs,axis=2),bins=range(10))
    plt.xticks(np.arange(0,10))
    plt.xlim(0,10)
    plt.xlabel("Accuracy of the prediction of the test digit")
    plt.ylabel("Frequency")
        

ImportError: cannot import name 'set_shapes_for_outputs'